In [1]:
# install dependencies: geopandas and kepler.gl
import sys
!conda install --yes --prefix {sys.prefix} geopandas
!{sys.executable} -m pip install keplergl
""

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

# All requested packages already installed.



''

In [2]:
import geopandas as gpd
from keplergl import KeplerGl
import pandas as pd

# what you still need to do manually:
This script can be used to display your google maps timeline data in a kepler.gl interactive map.
**However, you still need to download your timeline from google manually.**
To do that, just head over to http://takeout.google.com, uncheck everything except the *Location History* (which should be in json format per default) and click download archive at the bottom.
Then extract the zip file and move the *Location History.json* file to the project directory.

In [3]:
# Download convertmapstimeline.py script to convert maps timeline to geojson file
!wget https://raw.githubusercontent.com/nubenum/maps-timeline/master/convertmapstimeline.py
!{sys.executable} convertmapstimeline.py --geojson Location\ History.json locationHistory.json

--2019-07-07 15:55:24--  https://raw.githubusercontent.com/nubenum/maps-timeline/master/convertmapstimeline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7374 (7.2K) [text/plain]
Saving to: ‘convertmapstimeline.py.1’

convertmapstimeline 100%[===================>]   7.20K  --.-KB/s    in 0s      

2019-07-07 15:55:24 (26.8 MB/s) - ‘convertmapstimeline.py.1’ saved [7374/7374]



In [4]:
# load the geojson file and give a quick insight of what is in there
df = gpd.read_file('locationHistory.json')
df.head()

,geometry
0,"LINESTRING (13.6674999 48.0039763, 13.6609231 ..."
1,"LINESTRING (13.7225715 48.0162288, 13.7123034 ..."
2,"LINESTRING (13.7427447 48.0340214, 13.7427447 ..."
3,"LINESTRING (13.7664133 48.0504174, 13.760747 4..."
4,"LINESTRING (13.7693721 48.0524162, 13.7664133 ..."


# modifying the data:
We now imported the data into a geopandas dataframe, but now let's see what every line really does look like:

In [5]:
# number of records: 
print(len(df))
# select the first row
row = df.iloc[0]
line = row[0]
print(type(line))

13470
<class 'shapely.geometry.linestring.LineString'>


so we have a total of ... lines, 
each of them contains a lineString which consists of many points. This also now raises the question how many indivudial points we saved.
More detailed, we are interested in the pairs of two locations (which we can pass to kepler to draw arcs)

In [6]:
df['coords'] = df['geometry'].apply(lambda x: x.coords)

print("\nnumber of total coordinates in data:")
print(df['coords'].apply(lambda x: len(x)).sum())


number of total coordinates in data:
237675


# create coordinate pairs
To be able to draw arcs or lines in kepler.gl we want to convert the data into simple pairs of coordinates. Therefore we split every single LineString from the dataFrame into multiple pairs of coordinates.

Next up we split the coordinate pairs again into individual lat and long coordinate for arc start and end points.

In [7]:
def pairs(lst):
    for i in range(1, len(lst)):
        yield lst[i-1], lst[i]
        
def flatmap(self, func):
    rows = []
    for idx, row in self.iterrows():
        multrows = func(row)
        rows.extend(multrows)
    return pd.DataFrame.from_records(rows)

pd.DataFrame.flatmap = flatmap

# split lineStrings into coordinate pairs and create new DataFrame for them
pairs = pd.DataFrame(df.flatmap(lambda x: pairs(x['coords'])))

# extract individual coordinate floats from coordinate pairs
pairs['fromLat'] = pairs[0].apply(lambda c: c[0])
pairs['fromLong'] = pairs[0].apply(lambda c: c[1])
pairs['toLat'] = pairs[1].apply(lambda c: c[0])
pairs['toLong'] = pairs[1].apply(lambda c: c[1])

pairs.head()

,0,1,fromLat,fromLong,toLat,toLong
0,"(13.6674999, 48.0039763)","(13.6609231, 48.003433)",13.667500,48.003976,13.660923,48.003433
1,"(13.7225715, 48.0162288)","(13.7123034, 48.0062724)",13.722572,48.016229,13.712303,48.006272
2,"(13.7123034, 48.0062724)","(13.6674999, 48.0039763)",13.712303,48.006272,13.667500,48.003976
3,"(13.7427447, 48.0340214)","(13.7427447, 48.0340214)",13.742745,48.034021,13.742745,48.034021
4,"(13.7427447, 48.0340214)","(13.7225715, 48.0162288)",13.742745,48.034021,13.722572,48.016229


# HexBin map of starting points
The first map is a hexbin map of all the starting points of the trips. They are binned in 30 kilometer bins and shown on a three-dimensional map.

In [8]:
map1 = KeplerGl(height=700, data = {'data1': pairs.to_csv()}, config={'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'v65eirr',
     'type': 'hexagon',
     'config': {'dataId': 'data1',
      'label': 'Trip Starting Points',
      'color': [246, 209, 138],
      'columns': {'lat': 'fromLong', 'lng': 'fromLat'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'worldUnitSize': 30,
       'resolution': 8,
       'colorRange': {'name': 'ColorBrewer YlGn-5',
        'type': 'sequential',
        'category': 'ColorBrewer',
        'colors': ['#006837', '#31a354', '#78c679', '#c2e699', '#ffffcc'],
        'reversed': True},
       'coverage': 1,
       'sizeRange': [0, 500],
       'percentile': [0, 100],
       'elevationPercentile': [0, 100],
       'elevationScale': 10,
       'colorAggregation': 'count',
       'sizeAggregation': 'count',
       'enable3d': True},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'data1': ['column_0',
       'fromLong',
       'fromLat',
       'toLong',
       'toLat'],
      'data2': ['column_0', 'fromLong', 'fromLat', 'fromLong', 'toLat']},
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': []},
  'mapState': {'bearing': -2.1319796954314754,
   'dragRotate': True,
   'latitude': 47.710321487255285,
   'longitude': 14.111189743396244,
   'pitch': 49.49392839240254,
   'zoom': 5.629385866035164,
   'isSplit': False},
  'mapStyle': {'styleType': 'light',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'mapStyles': {}}}})
map1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'v65eirr', 'type': …

# Travel Arc Map
For every single trip between to points an arc is drawn on a three-dimensional map.

In [9]:
map2 = KeplerGl(height=700, data = {'data1': pairs.to_csv()}, config={'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'faswyd',
     'type': 'arc',
     'config': {'dataId': 'data1',
      'label': 'data1',
      'color': [190, 48, 9],
      'columns': {'lat0': 'fromLong',
       'lng0': 'fromLat',
       'lat1': 'toLong',
       'lng1': 'toLat'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 1,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 10],
       'targetColor': [254, 179, 26]},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'id': '0vf1x26',
     'type': 'geojson',
     'config': {'dataId': 'data1',
      'label': 'data1',
      'color': [255, 203, 153],
      'columns': {'geojson': '1'},
      'isVisible': False,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled': False,
       'enable3d': False,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'data1': ['column_0',
       'fromLong',
       'fromLat',
       'toLong',
       'toLat']},
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': []},
  'mapState': {'bearing': -15.654822335025386,
   'dragRotate': True,
   'latitude': 48.65014418291175,
   'longitude': 13.396759198585784,
   'pitch': 48.79907030563581,
   'zoom': 4.939065787978216,
   'isSplit': False},
  'mapStyle': {'styleType': 'light',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'mapStyles': {}}}})
map2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'faswyd', 'type': '…

# Finalization:
Last but not least, we will save our paired data to the *trips.csv* file and export our maps as html.

In [10]:
pairs.to_csv(r'data.csv')
map1.save_to_html(file_name='hexbins.html', read_only=True)
map2.save_to_html(file_name='arcs.html', read_only=True)

Map saved to hexbins.html!
Map saved to arcs.html!
